In [1]:
!pip install tcia_utils
!pip install pandas
!pip install numpy
!pip install openpyxl


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import sys

# install simpleDicomViewer and forked pydicom-seg dependency
!{sys.executable} -m pip install --upgrade -q git+https://github.com/kirbyju/pydicom-seg.git@master
!{sys.executable} -m pip install --upgrade -q simpleDicomViewer


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# import the necessary dependencies
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
from tqdm import tqdm
from tcia_utils import nbia
from simpleDicomViewer import dicomViewer

### Download the dcm files using the tcia API

In [4]:
data = nbia.getSeries(collection='NSCLC-Radiomics')

2025-05-23 12:37:20,635:INFO:Success - Token saved to global api_call_headers variable and expires at 2025-05-23 14:37:20.635790
2025-05-23 12:37:20,637:INFO:Accessing public data anonymously. To access restricted data use nbia.getToken() with your credentials.
2025-05-23 12:37:20,639:INFO:Calling getSeries with parameters {'Collection': 'NSCLC-Radiomics'}


In [5]:
df = nbia.getSeries(collection = "NSCLC-Radiomics", format = "df")
df.head(2)

2025-05-23 12:37:28,195:INFO:Calling getSeries with parameters {'Collection': 'NSCLC-Radiomics'}


,SeriesInstanceUID,StudyInstanceUID,Modality,ProtocolName,SeriesDate,BodyPartExamined,SeriesNumber,Collection,PatientID,Manufacturer,...,ImageCount,TimeStamp,LicenseName,LicenseURI,CollectionURI,FileSize,DateReleased,StudyDate,ThirdPartyAnalysis,SeriesDescription
0,1.3.6.1.4.1.32722.99.99.3208985276719002650392...,1.3.6.1.4.1.32722.99.99.2923185306557238260155...,CT,MAASTRO_PETCT_WholeBodyC,2006-04-13 00:00:00.0,LUNG,0,NSCLC-Radiomics,LUNG1-048,SIEMENS,...,135,2020-10-07 14:01:08.0,Creative Commons Attribution-NonCommercial 3.0...,https://creativecommons.org/licenses/by-nc/3.0/,https://doi.org/10.7937/K9/TCIA.2015.PF0M9REI,71041260,2020-10-07 14:01:08.0,2006-04-13 00:00:00.0,NO,NaN
1,1.3.6.1.4.1.32722.99.99.7162165312520158212424...,1.3.6.1.4.1.32722.99.99.1008555718320741529516...,CT,MAASTRO_PETCT_WholeBodyC,2006-01-06 00:00:00.0,LUNG,0,NSCLC-Radiomics,LUNG1-065,SIEMENS,...,135,2020-10-07 14:00:50.0,Creative Commons Attribution-NonCommercial 3.0...,https://creativecommons.org/licenses/by-nc/3.0/,https://doi.org/10.7937/K9/TCIA.2015.PF0M9REI,71040974,2020-10-07 14:00:50.0,2006-01-06 00:00:00.0,NO,NaN


In [6]:
df.columns

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'Modality', 'ProtocolName',
       'SeriesDate', 'BodyPartExamined', 'SeriesNumber', 'Collection',
       'PatientID', 'Manufacturer', 'ManufacturerModelName',
       'SoftwareVersions', 'ImageCount', 'TimeStamp', 'LicenseName',
       'LicenseURI', 'CollectionURI', 'FileSize', 'DateReleased', 'StudyDate',
       'ThirdPartyAnalysis', 'SeriesDescription'],
      dtype='object')

In [7]:
df["Modality"].value_counts()

Modality
CT          422
RTSTRUCT    422
SEG         421
Name: count, dtype: int64

In [8]:
seg_df = df[df['Modality'] == "SEG"]

In [9]:
series = seg_df["SeriesInstanceUID"]

In [10]:
series_data = series.to_dict()

In [11]:
converted = [{"SeriesInstanceUID": value} for value in series_data.values()]

In [ ]:
nbia.downloadSeries(
    series_data = converted
)

2025-05-23 09:55:47,046:INFO:Downloading 421 out of 421 Series Instance UIDs (scans).
2025-05-23 09:55:47,047:INFO:Directory 'tciaDownload' created successfully.
2025-05-23 09:55:47,048:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.2.276.0.7230010.3.1.3.2323910823.11644.1597260534.485
2025-05-23 09:55:48,157:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.2.276.0.7230010.3.1.3.2323910823.20524.1597260509.554
2025-05-23 09:55:49,246:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.2.276.0.7230010.3.1.3.2323910823.11436.1597260561.739
2025-05-23 09:55:50,292:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.2.276.0.7230010.3.1.3.2323910823.23864.1597260522.316
2025-05-23

### load the radiomics features

In [4]:
radiomics = pd.read_excel("radiomics.xlsx")

In [5]:
radiomics.head()

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event,...,wavelet-LLL_gldm_GrayLevelNonUniformity,wavelet-LLL_gldm_GrayLevelVariance,wavelet-LLL_gldm_HighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceEmphasis,wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LLL_gldm_LowGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceEmphasis,wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis
0,LUNG1-001,78.7515,2.0,3,0,IIIb,large cell,male,2165,1,...,13393.763031,179.264900,4832.419014,72.065403,395108.830811,0.017872,0.000442,0.189049,642.147174,0.000123
1,LUNG1-002,83.8001,2.0,0,0,I,squamous cell carcinoma,male,155,1,...,30435.593374,231.483934,4558.990657,118.826078,615187.351196,0.027186,0.000433,0.125072,371.860025,0.000096
2,LUNG1-003,68.1807,2.0,3,0,IIIb,large cell,male,256,1,...,1688.135276,218.269133,2577.313647,44.740697,149388.090307,0.021025,0.001081,0.273498,506.112016,0.000375
3,LUNG1-004,70.8802,2.0,1,0,II,squamous cell carcinoma,male,141,1,...,5660.448046,120.694667,4381.760740,61.175351,299133.133717,0.013747,0.000393,0.181132,613.881181,0.000125
4,LUNG1-005,80.4819,4.0,2,0,IIIb,squamous cell carcinoma,male,353,1,...,5794.013365,124.385590,5127.269539,70.726556,392174.428273,0.013114,0.000268,0.161437,642.519431,0.000082


In [14]:
radiomics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Columns: 1228 entries, PatientID to wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis
dtypes: float64(1220), int64(4), object(4)
memory usage: 4.0+ MB


### Extract the features

In [15]:
import os

dir_path = 'tciaDownload'
file_count = len([f for f in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, f))])

print(f"Number of files in '{dir_path}':", file_count)

Number of files in 'tciaDownload': 421


In [16]:
# code for extraction
def extract_tumor_xyz_from_seg(seg_path):
    print(f"Reading segmentation file: {seg_path}")
    image = sitk.ReadImage(seg_path)
    array = sitk.GetArrayFromImage(image)

    unique_labels = np.unique(array)
    print(f"Unique labels in segmentation: {unique_labels}")

    if len(unique_labels) <= 1:
        raise ValueError("Segmentation does not contain labeled voxels.")

    label = 1 if 1 in unique_labels else int(unique_labels[1])
    tumor_voxels = np.argwhere(array == label)

    if tumor_voxels.size == 0:
        raise ValueError(f"No voxels found with label: {label}")

    spacing = image.GetSpacing()
    origin = image.GetOrigin()
    direction = np.array(image.GetDirection()).reshape(3, 3)

    coords = []
    for z, y, x in tumor_voxels:
        phys = origin + direction @ (np.array([x, y, z]) * spacing)
        coords.append(phys)

    coords = np.array(coords)
    centroid = coords.mean(axis=0)
    min_xyz = coords.min(axis=0)
    max_xyz = coords.max(axis=0)

    return {
        "x": centroid[0],
        "y": centroid[1],
        "z": centroid[2],
        "min_x": min_xyz[0],
        "min_y": min_xyz[1],
        "min_z": min_xyz[2],
        "max_x": max_xyz[0],
        "max_y": max_xyz[1],
        "max_z": max_xyz[2]
    }

In [17]:
# --- Main Logic ---
dir_path = 'tciaDownload'
subdirs = [f for f in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, f))]

# Collect all .dcm file paths with corresponding subdir info
dcm_paths = []
for subdir in subdirs:
    full_subdir_path = os.path.join(dir_path, subdir)
    dcm_files = [f for f in os.listdir(full_subdir_path) if f.endswith('.dcm')]
    dcm_paths.extend([(os.path.join(full_subdir_path, f)) for f in dcm_files])

results = []

# Process with progress bar
for seg_path in tqdm(dcm_paths, desc="Processing DICOM files"):
    try:
        # Extract features
        features = extract_tumor_xyz_from_seg(seg_path)

        # Read PatientID
        image = sitk.ReadImage(seg_path)
        patient_id = image.GetMetaData("0010|0020") if image.HasMetaDataKey("0010|0020") else "Unknown"

        # Store as a row
        row = {
            "PatientID": patient_id,
            **features
        }

        results.append(row)

    except Exception as e:
        print(f"❌ Failed processing {seg_path}: {e}")

Processing DICOM files:   0%|          | 0/421 [00:00<?, ?it/s]

Reading segmentation file: tciaDownload/1.2.276.0.7230010.3.1.3.2323910823.24656.1597260676.393/1-1.dcm


Unique labels in segmentation: [  0 255]


Processing DICOM files:   0%|          | 1/421 [00:35<4:05:35, 35.09s/it]

Reading segmentation file: tciaDownload/1.2.276.0.7230010.3.1.3.2323910823.7812.1597259950.739/1-1.dcm
Unique labels in segmentation: [  0 255]


...

Processing DICOM files: 100%|█████████▉| 419/421 [1:54:05<00:28, 14.49s/it]

Reading segmentation file: tciaDownload/1.2.276.0.7230010.3.1.3.2323910823.25268.1597260096.416/1-1.dcm
Unique labels in segmentation: [  0 255]


Processing DICOM files: 100%|█████████▉| 420/421 [1:54:22<00:15, 15.36s/it]

Reading segmentation file: tciaDownload/1.2.276.0.7230010.3.1.3.2323910823.24516.1597257820.582/1-1.dcm
Unique labels in segmentation: [  0 255]


Processing DICOM files: 100%|██████████| 421/421 [1:54:47<00:00, 16.36s/it]


In [18]:
# Save to CSV
df = pd.DataFrame(results)
df.to_csv("centroid.csv", index=False)

print(f"✅ Completed processing {len(results)} DICOM files.")
print("💾 Results saved to centroid.csv")

✅ Completed processing 421 DICOM files.
💾 Results saved to centroid.csv


In [ ]:
# Load the tumor segmentation summary
tumor_df = pd.read_csv("centroid.csv")

# Strip whitespace from PatientID columns in both DataFrames
radiomics['PatientID'] = radiomics['PatientID'].astype(str).str.strip()
tumor_df['PatientID'] = tumor_df['PatientID'].astype(str).str.strip()

# Merge on PatientID
merged_df = radiomics.merge(tumor_df, on="PatientID", how="left")

# Save the result
merged_df.to_csv("radiomic_features.csv", index=False)

print("✅ Merge complete. Saved to merged_patient_data.csv")

✅ Merge complete. Saved to merged_patient_data.csv


In [11]:
radiomics.head()

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event,...,wavelet-LLL_gldm_GrayLevelNonUniformity,wavelet-LLL_gldm_GrayLevelVariance,wavelet-LLL_gldm_HighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceEmphasis,wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LLL_gldm_LowGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceEmphasis,wavelet-LLL_gldm_SmallDependenceHighGrayLevelEmphasis,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis
0,LUNG1-001,78.7515,2.0,3,0,IIIb,large cell,male,2165,1,...,13393.763031,179.264900,4832.419014,72.065403,395108.830811,0.017872,0.000442,0.189049,642.147174,0.000123
1,LUNG1-002,83.8001,2.0,0,0,I,squamous cell carcinoma,male,155,1,...,30435.593374,231.483934,4558.990657,118.826078,615187.351196,0.027186,0.000433,0.125072,371.860025,0.000096
2,LUNG1-003,68.1807,2.0,3,0,IIIb,large cell,male,256,1,...,1688.135276,218.269133,2577.313647,44.740697,149388.090307,0.021025,0.001081,0.273498,506.112016,0.000375
3,LUNG1-004,70.8802,2.0,1,0,II,squamous cell carcinoma,male,141,1,...,5660.448046,120.694667,4381.760740,61.175351,299133.133717,0.013747,0.000393,0.181132,613.881181,0.000125
4,LUNG1-005,80.4819,4.0,2,0,IIIb,squamous cell carcinoma,male,353,1,...,5794.013365,124.385590,5127.269539,70.726556,392174.428273,0.013114,0.000268,0.161437,642.519431,0.000082


In [10]:
tumor_df.head()

,PatientID,x,y,z,min_x,min_y,min_z,max_x,max_y,max_z
0,LUNG1-022,60.190313,14.599448,335.368303,-74.1120,-92.1120,-75.9,181.8880,114.8880,634.1
1,LUNG1-174,-53.208900,-195.329511,-378.726778,-125.5117,-299.5117,-621.0,49.4883,-99.5117,-228.0
2,LUNG1-122,3.406338,6.853500,-408.164315,-146.5117,-101.5117,-602.5,157.4883,97.4883,-169.5
3,LUNG1-411,9.138102,-180.240372,-325.217594,-134.5117,-287.5117,-639.0,137.4883,-91.5117,-41.0
4,LUNG1-359,10.556907,-157.485831,-294.473415,-126.5117,-243.5117,-709.0,140.4883,-75.5117,-87.0


In [27]:
merged_df.head()

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event,...,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis,x,y,z,min_x,min_y,min_z,max_x,max_y,max_z
0,LUNG1-001,78.7515,2.0,3,0,IIIb,large cell,male,2165,1,...,0.000123,6.196296,-184.722568,-413.653013,-139.5117,-300.5117,-616.5,166.4883,-81.5117,-169.5
1,LUNG1-002,83.8001,2.0,0,0,I,squamous cell carcinoma,male,155,1,...,0.000096,16.448367,-9.154733,251.858923,-126.1120,-105.1120,-121.4,152.8880,185.8880,531.6
2,LUNG1-003,68.1807,2.0,3,0,IIIb,large cell,male,256,1,...,0.000375,13.741307,-34.750982,73.283886,-124.1120,-152.1120,-299.9,152.8880,82.8880,335.1
3,LUNG1-004,70.8802,2.0,1,0,II,squamous cell carcinoma,male,141,1,...,0.000125,4.155463,13.561335,-307.463080,-137.5117,-106.5117,-624.0,165.4883,123.4883,-97.0
4,LUNG1-005,80.4819,4.0,2,0,IIIb,squamous cell carcinoma,male,353,1,...,0.000082,-6.221136,-17.284232,138.648683,-152.1120,-130.1120,-168.4,124.8880,96.8880,365.6


In [32]:
merged_df[merged_df["x"].isnull()]

,PatientID,age,clinical.T.Stage,Clinical.N.Stage,Clinical.M.Stage,Overall.Stage,Histology,gender,Survival.time,deadstatus.event,...,wavelet-LLL_gldm_SmallDependenceLowGrayLevelEmphasis,x,y,z,min_x,min_y,min_z,max_x,max_y,max_z
127,LUNG1-128,64.5859,3.0,1,0,IIIa,nos,female,335,0,...,0.000239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
